<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Contents](Index.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.03-Operations-in-Pandas.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# 在Pandas中操作資料

Pandas跟Numpy一樣，可以快速地進行元素的簡單(加減乘除)和複雜(三角函數)的運算

同時，Pandas包含了2個變形:
1. 對於負數和三角函數的單元運算，ufuncs可以在輸出時*保留索引和欄標籤* 
2. 二元操作(像是加減法)當把物件傳遞給ufuncs時，Pandas會自動*對齊索引*。

代表說要從不同的資料來源保存資料內容和合併資料(包含潛在對於Numpy陣列中進行錯誤移除工作)，在Pands中會是非常簡單的事情

在此將更進一步檢視在一維``Series``結構和二維``DataFrame``結構間的操作.

## Ufuncs: 保存索引

因為Pandas是設計來和NumPy搭配使用的, 任何NumPy的ufunc都可以在 Pandas的``Series``和``DataFrame``物件上操作。


In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


如果應用Numpy的ufunc在這些物件上，結果將會是另外一個*被保留索引*的Pandas物件

In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## UFuncs: 對齊索引

對於在兩個``Series``或``DataFrame``物件間的二元運算，Pandas會在執行運算的過程中對齊索引。

若是在不完整的資料中工作時，這將會非常方便。

### 在Series中對齊索引

舉例: 合併面積和人口資料，且找出均前3名的州

In [6]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

In [7]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

若有缺失值，會以``NaN``, or "Not a Number,"表示

In [8]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

若使用NaN不是你想要的, 可以在運算子的地方使用適當的物件方法去填入資料。

例如: 呼叫``A.add(B)``相等於``A + B``, 但是允許明確的指定，當任一元素的資料遺失時要填入的特定值:

In [9]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### 在DataFrame中的索引對齊

當在``DataFrame``上執行運算時，同樣型式的對齊也會同時發生在欄和索引值上:

In [17]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,8,6
1,17,3


In [18]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,8,1,9
1,8,9,4
2,1,3,6


In [19]:
A + B

,A,B,C
0,9.0,14.0,NaN
1,26.0,11.0,NaN
2,NaN,NaN,NaN


注意: 索引被正確的對齊和它們在2個物件之內的順序無關，而且在結果中的索引會被排序。就像之前在``Series``中的例子, 可以使用相關聯的物件算術方法，傳遞任何想要的``fill_value``放在缺失項目的位置上。
於此，將在``A``中填入所有值的平均數 (先堆疊所有的列再計算平均值):

In [21]:
fill = A.stack().mean()# 8.5
A.add(B, fill_value=fill)

,A,B,C
0,9.0,14.0,17.5
1,26.0,11.0,12.5
2,11.5,9.5,14.5


在Python運算子和Pandas方法之間的對應:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: 在DataFrame和Series間的操作

當你在``DataFrame``和``Series``間執行運算時, 索引和欄都是使用相同的方式對齊。在``DataFrame``和``Series``間的運算就好像是在Numpy中的二維和一維操作一樣。 

In [22]:
# 找出一個二維陣列和它其中一列的差:
A = rng.randint(10, size=(3, 4))
A

array([[7, 2, 0, 3],
       [1, 7, 3, 1],
       [5, 5, 9, 3]])

In [23]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-6,  5,  3, -2],
       [-2,  3,  9,  0]])

根據Numpy的Broadcasting規則，在一個二維陣列上減去它的一個列會被逐列套用。

在Pandas,類似的慣例也是預設以逐列的方式操作:

In [24]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-6,5,3,-2
2,-2,3,9,0


In [25]:
df

,Q,R,S,T
0,7,2,0,3
1,1,7,3,1
2,5,5,9,3


In [26]:
# 逐欄(axis = 0)
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,5,0,-2,1
1,-6,0,-4,-6
2,0,0,4,-2


``DataFrame``/``Series``運算，會自動在2個元素間對齊索引:

In [ ]:
halfrow = df.iloc[0, ::2]
halfrow

Q    6
S    2
Name: 0, dtype: int64

<!--NAVIGATION-->
< [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Contents](Index.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.03-Operations-in-Pandas.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
